# Lab 1. Forecasting Agent

## Introduction

In this notebook we show you how to create your first sub-agent on [Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/).

Amazon Bedrock Agents enable generative AI applications to execute multi-step business tasks using natural language.

In our first example we will create a forecasting agent, where customers can ask the agent to return information about their current energy consumption and forecast of it. 

The following represents the piece of architecture that will be built on this module.
<img src="img/forecast_agent.png" class="img-responsive" alt=""> </div>


In this example, we will also enable our agent to use code intepretation capabilities to plot energy usage and its forecast. We are also using [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/) to provide documentation about the forecasting model and its capabilities

## Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure to have version superior of **1.35.45**

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install Dependencies from local package
!pip install ../boto3/controlplane/botocore-1.35.55-py3-none-any.whl \
    ../boto3/controlplane/boto3-1.35.55-py3-none-any.whl \
    ../boto3/controlplane/awscli-1.35.21-py3-none-any.whl --force-reinstall --no-cache

In [ ]:
# Only install if running locally
!pip install pickleshare --force-reinstall --no-cache

One component of a Knowledge Base is a Vector Database. Vector database is a kind of database that is optimized to store and search with vectors, making RAG architectures possible and faster. 

Amazon Bedrock Knowledge Bases is powered by one vector database. 

For this labwe are going to create one collection forAmazon OpenSearch Serverless. If you want to know more, [this link](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-setup.html) contains other supported vector engines for Amazon Bedrock Knowledge Bases.

Now, let's install opensearch-py to interact with Opensearch engine.

In [ ]:
!pip install opensearch-py retrying --force-reinstall --no-cache

Restart kernel to ensure packages updates to take effect

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

Check if your boto3 version is superior than **1.35.45**

In [ ]:
!pip freeze | grep boto3

## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and you will start to create your first agent.

In [ ]:
import boto3
import os
import json
import sys
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

In [ ]:
forecast_agent_name = f"forecast-{agent_suffix}"

forecast_lambda_name = f"fn-forecast-agent-{agent_suffix}"

forecast_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{forecast_agent_name}'

dynamodb_table = f"{forecast_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "day"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{forecast_agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing information on how forecasting process is done"
bucket_name = f'{forecast_agent_name}-{suffix}'


On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the file can be recognized and invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(3, "../../../..")

from src.utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from src.utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and Load Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

This data contains basic information about how forecast process is done.

**This creation process takes around 5min.**

In [ ]:
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

## Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [ ]:
path = "kb_documents"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [ ]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            { "text": prompt } 
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 500,
            "temperature": 0
        },
    )

    return response['output']['message']

def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close

### Generating data prompt
Generating one file with forecasting info using the LLM model

In [ ]:
text_generation_energy_instructions = '''
    You will be act as data-scientist that knows forecasting, Python and scikit learn.
    You will generate a step-by-step on how to create a forecast process for a time-series data.

    This data has the following json structure:
    {"customer_id": "1", "day": "2024/06/01", "sumPowerReading": "120.0", "kind":"measured"}

    Choose one forecast algorithm, preferable one that works on scikit-learn, and explain 
      detailed how to create a step by step, with code sample, to run forecast on this data.

    Answer only with the step-by-step, avoid answer with afirmations like: OK, I can generate it, or 
      Yes, please find following example. Be direct and only reply the step-by-step.
'''

solar_energy_file_name = 'forecasting-info.txt'

response_message = invoke_bedrock_generate_energy_files(text_generation_energy_instructions)

write_file('{}/{}'.format(path, solar_energy_file_name),response_message['content'][0]['text'])

### Uploading data to s3
Uploading generated files into an Amazon S3 Bucket.

In [ ]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [ ]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

## Creating Agent

Create the forecast agent that will have a Knowledge Base with information on how forecast process is done as well as the action groups to handle the user requests and `code interpretation` capabilities for plotting the graphs to fulfill user requests.

In order to have accurate agents, it is important to set unambiguous instructions of what the agent should do and what it should not do. It is also important to provide clear definitions for when the agent should use the knowledge bases and action groups available to it.

We will provide the following instructions to our agent:
```
You are an Energy Assistant that helps customers understand their energy consumption patterns and future usage expectations.

Your capabilities include:
1. Analyzing historical energy consumption
2. Providing consumption forecasts
3. Generating usage statistics
4. Updating forecasts for specific customers

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner

Visualization capabilities:
- You may only create graphs and charts specifically related to:
  * Energy consumption data
  * Energy forecast visualization
- Decline any other visualization or code generation requests politely

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- Do not add extra information not required by the user
```

We will also connect a knowledge base for the explanation of the forecasting methodologies with the following instructions:
```
Access this knowledge base when needing to explain specific forecast generation methodology.
```

And we will make the following tool available to the agent:
- `get_forecasted_consumption`: Gets the next 3 months energy usage forecast
- `get_historical_consumption`: Gets energy usage history to date
- `get_consumption_statistics`: Gets current month usage analytics
- `update_forecasting`: Updates the energy forecast for a specific month


In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Access this knowledge base when needing to explain specific forecast generation methodology."""
}

In [ ]:
agent_description = """You are a energy usage forecast bot.
You can retrieve historical energy consumption, forecasted consumption, usage statistics and update a forecast for a specific user"""

agent_instruction = """You are an Energy Assistant that helps customers understand their energy consumption patterns and future usage expectations.

Your capabilities include:
1. Analyzing historical energy consumption
2. Providing consumption forecasts
3. Generating usage statistics
4. Updating forecasts for specific customers

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner

Visualization capabilities:
- You may only create graphs and charts specifically related to:
  * Energy consumption data
  * Energy forecast visualization
- Decline any other visualization or code generation requests politely

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user"""

forecast_agent = agents.create_agent(
    forecast_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=True
)

forecast_agent

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [ ]:
agents.associate_kb_with_agent(
    forecast_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

### Creating Lambda

In order to enable the agent to execute tasks, we will create an AWS Lambda function that implements the tasks execution. We will then provide this lambda function to the agent action group. You can find more information on how to use action groups to define actions that your agent can perform [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html)

On this block, we're going to generate Lambda function Code:

In [ ]:
%%writefile forecast.py
import boto3
import json
import os

from boto3.dynamodb.conditions import Key, Attr
from datetime import datetime
from decimal import Decimal

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')
truncated_month = datetime.today().replace(day=1, hour=0, minute=0, second=0, microsecond=0)


def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def trunc_datetime(month,year):
    return datetime.today().replace(year =int(year), month=int(month), day=1, hour=0, minute=0, second=0, microsecond=0)

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(
    table_name: str, 
    pk_field: str,
    pk_value: str,
    sk_field: str=None, 
    sk_value: str=None,
    attr_key: str=None,
    attr_val: str=None
):
    try:

        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_field:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).eq(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        if attr_key:
            attr_expression = Attr(attr_key).eq(attr_val)
            query_data = table.query(
                KeyConditionExpression=key_expression,
                FilterExpression=attr_expression
            )
        else:
            query_data = table.query(
                KeyConditionExpression=key_expression
            )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def get_forecasted_consumption(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         attr_key="kind", attr_val="forecasted")

def get_historical_consumption(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         attr_key="kind", attr_val="measured")

def get_consumption_statistics(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         dynamodb_sk, 
                         truncated_month.strftime('%Y/%m/%d'))

def update_forecasting(customer_id, month, year, usage):
    current_date = trunc_datetime(month, year)
    if  current_date >= truncated_month:
        item = {
            'customer_id': customer_id,
            'day': current_date.strftime('%Y/%m/%d'),
            'sumPowerReading': Decimal(usage),
            'kind': 'forecasted'
        }
        put_dynamodb(dynamodb_table, item)
        return "Day: {} updated for customer: {}".format(current_date.strftime('%Y/%m/%d'), customer_id)
    else:
        return "You're trying to change a past date: {} for customer: {}, which is not allowed".format(current_date.strftime('%Y/%m/%d'), customer_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'get_forecasted_consumption':
        result = get_forecasted_consumption(customer_id)
    elif function == 'get_historical_consumption':
        result = get_historical_consumption(customer_id)
    elif function == 'get_consumption_statistics':
        result = get_consumption_statistics(customer_id)
    elif function == 'update_forecasting':
        month = get_named_parameter(event, "month")
        year = get_named_parameter(event, "year")
        usage = get_named_parameter(event, "usage")
        result = update_forecasting(customer_id, month, year, usage)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

### Defining available actions

Next we will define the available actions that an agent can perform using [Function Details](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html). You can also do this task using OpenAPI Schemas, which can be very useful if you already have an OpenAPI schema available for your application.

When creating your function details, it is important to provide clear descriptions for the function and for its parameters, as your agent depends on them to correctly orchestrate the tasks to be executed

In [ ]:
functions_def = [
    {
        "name": "get_forecasted_consumption",
        "description": """Gets the next 3 months energy usage forecast""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_historical_consumption",
        "description": """Gets energy usage history to date""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_consumption_statistics",
        "description": """Gets current month usage analytics""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "update_forecasting",
        "description": """Updates the energy forecast for a specific month""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "month": {
                "description": "Target update month. In the format MM",
                "required": True,
                "type": "integer"
            },
            "year": {
                "description": "Target update year. In the format YYYY",
                "required": True,
                "type": "integer"
            },
            "usage": {
                "description": "New consumption value",
                "required": True,
                "type": "integer"
            }
        }
    }
]

### Creating action group and attaching to the agent
Now it's time to add this Lambda function and the function details as an action group for this agent and prepare it.

In [ ]:
agents.add_action_group_with_lambda(
    agent_name=forecast_agent_name,
    lambda_function_name=forecast_lambda_name,
    source_code_file="forecast.py",
    agent_functions=functions_def,
    agent_action_group_name="forecast_consumption_actions",
    agent_action_group_description="Function to get usage forecast for a user ",
    dynamo_args=dynamoDB_args
)

In [ ]:
forecast_lambda_name

## Loading data to DynamoDB

Now that we've created our agent, let's load some generated data to DynamoDB. That will allow the agent to interact with some live data to perform actions

In [ ]:
with open("1_user_sample_data.json") as f:
    table_items = [json.loads(line) for line in f]

agents.load_dynamodb(dynamodb_table, table_items)

Testing that data was loaded on DynamoDB

In [ ]:
resp = agents.query_dynamodb(
    dynamodb_table, dynamodb_pk, '1', dynamodb_sk, "2024/06/01"
)
resp

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

### Testing get forecast
First let's test the get forecast action together with the code interpretation capabilities

In [ ]:
%%time
print(
    agents.invoke(
        "can you give me my forecasted energy consumption in a bar chart? My customer id is 1", 
        forecast_agent[0]
    )
)

### Testing get historical consumption
Now we can test the historical energy consumption and also use code interpretation to plot it in a bar chart

In [ ]:
time.sleep(60)

In [ ]:
%%time
print(
    agents.invoke(
        "can you give me my energy consumption in a bar chart? My customer id is 1", 
        forecast_agent[0]
    )
)

### Testing knowledge base access
Now let's check the knowledge base access by asking a question about the forecasting algorithm

In [ ]:
time.sleep(60)

In [ ]:
%%time
print(
    agents.invoke(
        "What's algorithm used for forecast?", 
        forecast_agent[0]
    )
)

### Testing forecast update
Now we can test the functionality to update the expected forecasting

In [ ]:
time.sleep(60)

In [ ]:
%%time
print(
    agents.invoke(
        "Can you update my forecast for month 12/2024? I will be travelling and my estimate will be 50. My id is 1", 
        forecast_agent[0]
    )
)

### Confirming that forecast was updated
After updating our forecast, let's check that the forecast was updated and plot a new graph

In [ ]:
time.sleep(60)

In [ ]:
%%time
print(
    agents.invoke(
        "can you give me my forecasted energy consumption in a bar chart? My id is 1", 
        forecast_agent[0]
    )
)

### Testing forecasting statistics
Finally, let's test the get stats functionality

In [ ]:
time.sleep(60)

In [ ]:
%%time
print(
    agents.invoke(
        "can you give me my current consumption? My id is 1", 
        forecast_agent[0]
    )
)

## Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
forecast_agent_alias_id, forecast_agent_alias_arn = agents.create_agent_alias(
    forecast_agent[0], 'v1'
)

## Saving information
Let's store some environment variables to be used on our next notebooks.

In [ ]:
forecast_agent_arn = agents.get_agent_arn_by_name(forecast_agent_name)
forecast_agent_id = forecast_agent[0]
forecast_kb = knowledge_base_name
forecast_dynamodb = dynamodb_table

%store forecast_agent_arn
%store forecast_agent_alias_arn
%store forecast_agent_alias_id
%store forecast_lambda_name
%store forecast_agent_name
%store forecast_agent_id
%store forecast_kb
%store forecast_dynamodb

In [ ]:
forecast_agent_arn, forecast_agent_alias_arn, forecast_agent_alias_id

## Next Steps
Congratulations! We've now created a forecasting agent. Next we will create our solar panel agent